# Linking DEC Metadata with J2SR Metrics

## Step 1: Import Packages

In [25]:
import pandas as pd
import numpy as np
import pycountry

## Step 2: Import DEC Data and Create Unique Country-Year ID

In [8]:
dec_meta = pd.read_csv('dec-evaluations-data.csv', index_col = "Unique_ID", encoding = 'latin')

In [20]:
dec_meta.head()

,Abstract,Ancillary_Data,Bibliographic_Type,ContentType,Contract_Grant_Number,Credit,Date_Resource_Created,Description,Descriptors_Topical,Descriptors_Geographic,...,Report_Number,Series_Title,Title,Title_Translated,URI,USAID_Geography,USAID_Project_Number,Digital_Object_Identifier,Date_Form,year
Unique_ID,,,,,,,,,,,,,,,,,,,,,
PD-AAB-495-E1,NaN,Evaluation period: 22 Mar 1972-12 Jul 1976,Final Evaluation Report,Documents,NaN,NaN,7/1/1977 0:00,NaN,Management education ~|~_©_~|~ Graduate educat...,Central America,...,NaN,NaN,LOAN COMPLETION REVIEW AND REPORT,NaN,https://dec.usaid.gov/dec/content/Detail.aspx?...,Reg Office Cent America & Panama (ROCAP),5960044,NaN,1977-07-01,1977.0
PD-AAP-604,NaN,Evaluation period: 25 Jul 1972-10 Apr 1979,Final Evaluation Report,Documents,NaN,NaN,6/1/1979 0:00,NaN,Low cost housing ~|~_©_~|~ Minimum shelter hou...,Peru,...,NaN,NaN,Urban reconstruction -- earthquake and flood z...,NaN,https://dec.usaid.gov/dec/content/Detail.aspx?...,Peru,5270101,NaN,1979-06-01,1979.0
PD-AAB-088-A1,NaN,NaN,Special Evaluation,Documents,AID/TA-C-1469,NaN,2/1/1979 0:00,NaN,Road construction ~|~_©_~|~ Roads ~|~_©_~|~ Ru...,Honduras,...,NaN,NaN,TRIALS SELECTION CRITERIA AND EVALUATION DESIG...,NaN,https://dec.usaid.gov/dec/content/Detail.aspx?...,Honduras,5220137,NaN,1979-02-01,1979.0
PD-AAA-880-D1,NaN,Evaluation period: 22 Jan 1979-9 Feb 1979,Final Evaluation Report,Documents,AID/LAC-C-1313,NaN,2/1/1979 0:00,NaN,Health surveys ~|~_©_~|~ Demographic research ...,El Salvador,...,NaN,NaN,EVALUATION OF MULTIPURPOSE HOUSEHOLD SURVEY (E...,NaN,https://dec.usaid.gov/dec/content/Detail.aspx?...,El Salvador,5190176,NaN,1979-02-01,1979.0
PD-AAT-461,NaN,Evaluation period: 1 Jan1972-31 Dec1976,Special Evaluation,Documents,AID/pha/C-1100,NaN,7/1/1977 0:00,NaN,Condoms ~|~_©_~|~ Sterilization (birth control...,Thailand,...,NaN,NaN,REPORT OF THE SECOND EVALUATION OF THE NATIONA...,NaN,https://dec.usaid.gov/dec/content/Detail.aspx?...,Thailand,4930283,NaN,1977-07-01,1977.0


In [13]:
dec_meta['Date_Form'] = pd.to_datetime(dec_meta['Date_Resource_Created'], infer_datetime_format = True)

In [37]:
dec_meta['year'] = pd.DatetimeIndex(dec_meta['Date_Form']).year.astype(str)
dec_meta['year'] = dec_meta['year'].str[:-2]
dec_meta['year']

Unique_ID
PD-AAB-495-E1    1977
PD-AAP-604       1979
PD-AAB-088-A1    1979
PD-AAA-880-D1    1979
PD-AAT-461       1977
                 ... 
PA-00S-RX6       2017
PA-00S-RV5       2016
PA-00S-RSS       2016
PA-00S-RRZ       2017
PA-00S-RRS       2016
Name: year, Length: 12411, dtype: object

In [26]:
def do_fuzzy_search(country):
    try:
        result = pycountry.countries.search_fuzzy(country)
    except Exception:
        return np.nan
    else:
        return result[0].alpha_3

In [27]:
iso_map = {country: do_fuzzy_search(country) for country in dec_meta["Descriptors_Geographic"].unique()}
dec_meta["country_code"] = dec_meta["Descriptors_Geographic"].map(iso_map)

In [28]:
dec_meta['country_code']

Unique_ID
PD-AAB-495-E1    NaN
PD-AAP-604       PER
PD-AAB-088-A1    HND
PD-AAA-880-D1    SLV
PD-AAT-461       THA
                ... 
PA-00S-RX6       NaN
PA-00S-RV5       NaN
PA-00S-RSS       IDN
PA-00S-RRZ       MDG
PA-00S-RRS       IDN
Name: country_code, Length: 12411, dtype: object

In [38]:
dec_meta['country_year'] = dec_meta['country_code'] + "_" + dec_meta['year']

In [39]:
dec_meta['country_year']

Unique_ID
PD-AAB-495-E1         NaN
PD-AAP-604       PER_1979
PD-AAB-088-A1    HND_1979
PD-AAA-880-D1    SLV_1979
PD-AAT-461       THA_1977
                   ...   
PA-00S-RX6            NaN
PA-00S-RV5            NaN
PA-00S-RSS       IDN_2016
PA-00S-RRZ       MDG_2017
PA-00S-RRS       IDN_2016
Name: country_year, Length: 12411, dtype: object

## Step 3: Import J2SR Data and Create Unique Country-Year ID

In [67]:
pd.options.display.max_colwidth = 100
j2sr_data = pd.read_csv('data_records.1598319212.csv')
j2sr_data.head()

,series_id,series_name,source_name,country_id,country_name,region,income_group,year,value
0,68092,"Roadmap Metric: Business Environment (0-100, higher is better)","USAID, Journey to Self-Reliance Metrics",4,Afghanistan,Asia,Low Income Country (World Bank Classification),2007,35.153241
1,68092,"Roadmap Metric: Business Environment (0-100, higher is better)","USAID, Journey to Self-Reliance Metrics",4,Afghanistan,Asia,Low Income Country (World Bank Classification),2008,34.844764
2,68092,"Roadmap Metric: Business Environment (0-100, higher is better)","USAID, Journey to Self-Reliance Metrics",4,Afghanistan,Asia,Low Income Country (World Bank Classification),2009,35.229759
3,68092,"Roadmap Metric: Business Environment (0-100, higher is better)","USAID, Journey to Self-Reliance Metrics",4,Afghanistan,Asia,Low Income Country (World Bank Classification),2010,42.153961
4,68092,"Roadmap Metric: Business Environment (0-100, higher is better)","USAID, Journey to Self-Reliance Metrics",4,Afghanistan,Asia,Low Income Country (World Bank Classification),2011,40.856806


In [68]:
j2sr_data['roadmap_metric'] = j2sr_data['series_name'].str.contains('Roadmap Metric:')
j2sr_data['roadmap_metric']

0        True
1        True
2        True
3        True
4        True
         ... 
49853    True
49854    True
49855    True
49856    True
49857    True
Name: roadmap_metric, Length: 49858, dtype: bool

In [69]:
j2sr_iso_map = {country: do_fuzzy_search(country) for country in j2sr_data["country_name"].unique()}
j2sr_data["country_code"] = j2sr_data["country_name"].map(iso_map)

In [70]:
j2sr_data['country_year'] = j2sr_data['country_code'] + "_" + j2sr_data['year'].astype(str)
j2sr_data['country_year']

0        AFG_2007
1        AFG_2008
2        AFG_2009
3        AFG_2010
4        AFG_2011
           ...   
49853    VNM_1995
49854    YEM_1995
49855    ZAF_1995
49856    ZMB_1995
49857    ZWE_1995
Name: country_year, Length: 49858, dtype: object

In [74]:
j2sr_roadmap = j2sr_data[j2sr_data['roadmap_metric']]
j2sr_roadmap

,series_id,series_name,source_name,country_id,country_name,region,income_group,year,value,roadmap_metric,country_code,country_year
0,68092,"Roadmap Metric: Business Environment (0-100, higher is better)","USAID, Journey to Self-Reliance Metrics",4,Afghanistan,Asia,Low Income Country (World Bank Classification),2007,35.153241,True,AFG,AFG_2007
1,68092,"Roadmap Metric: Business Environment (0-100, higher is better)","USAID, Journey to Self-Reliance Metrics",4,Afghanistan,Asia,Low Income Country (World Bank Classification),2008,34.844764,True,AFG,AFG_2008
2,68092,"Roadmap Metric: Business Environment (0-100, higher is better)","USAID, Journey to Self-Reliance Metrics",4,Afghanistan,Asia,Low Income Country (World Bank Classification),2009,35.229759,True,AFG,AFG_2009
3,68092,"Roadmap Metric: Business Environment (0-100, higher is better)","USAID, Journey to Self-Reliance Metrics",4,Afghanistan,Asia,Low Income Country (World Bank Classification),2010,42.153961,True,AFG,AFG_2010
4,68092,"Roadmap Metric: Business Environment (0-100, higher is better)","USAID, Journey to Self-Reliance Metrics",4,Afghanistan,Asia,Low Income Country (World Bank Classification),2011,40.856806,True,AFG,AFG_2011
...,...,...,...,...,...,...,...,...,...,...,...,...
49853,70295,Roadmap Metric: Export Sophistication,"USAID, Journey to Self-Reliance Metrics",704,Vietnam,Asia,Lower Middle Income Country (World Bank Classification),1995,-0.915000,True,VNM,VNM_1995
49854,70295,Roadmap Metric: Export Sophistication,"USAID, Journey to Self-Reliance Metrics",887,Yemen,Middle East and North Africa,Low Income Country (World Bank Classification),1995,-1.413300,True,YEM,YEM_1995
49855,70295,Roadmap Metric: Export Sophistication,"USAID, Journey to Self-Reliance Metrics",710,South Africa,Sub-Saharan Africa,Upper Middle Income Country (World Bank Classification),1995,0.314500,True,ZAF,ZAF_1995
49856,70295,Roadmap Metric: Export Sophistication,"USAID, Journey to Self-Reliance Metrics",894,Zambia,Sub-Saharan Africa,Lower Middle Income Country (World Bank Classification),1995,-0.602400,True,ZMB,ZMB_1995


In [72]:
j2sr_roadmap = j2sr_roadmap.dropna(subset=['country_year'])
j2sr_roadmap.shape

(35286, 12)

## Step 4: Pivot J2SR Data to Generate Columns per Indicator

In [81]:
j2sr_pivot = j2sr_roadmap.pivot_table(index = 'country_year', columns = 'series_name', values = 'value')
j2sr_pivot['iso'] = j2sr_pivot.index.str[:3]
j2sr_pivot['year'] = j2sr_pivot.index.str[4:]
j2sr_pivot

series_name,"Roadmap Metric: Biodiversity & Habitat Protections (0-100, higher is better)","Roadmap Metric: Business Environment (0-100, higher is better)","Roadmap Metric: Child Health (0-100, higher is better)","Roadmap Metric: Civil Society & Media Effectiveness (0-1, higher is better)","Roadmap Metric: Economic Gender Gap (0-1, where 0=inequality and 1=equality)",Roadmap Metric: Education Quality (years),Roadmap Metric: Export Sophistication,"Roadmap Metric: Information & Communication Technology (ICT) Adoption (1-100, higher is better)","Roadmap Metric: Liberal Democracy (0-1, higher is better)","Roadmap Metric: Open Government (0-1, higher is stronger rule of law)",Roadmap Metric: Poverty Rate ($5/Day) (percentage),"Roadmap Metric: Safety & Security (0-100, higher is better)","Roadmap Metric: Social Group Equality (0-4, higher is better)","Roadmap Metric: Tax System Effectiveness (0-1, higher is better)",iso,year
country_year,,,,,,,,,,,,,,,,
AFG_1950,NaN,NaN,NaN,0.320,NaN,NaN,NaN,NaN,0.085,NaN,NaN,NaN,0.974,NaN,AFG,1950
AFG_1951,NaN,NaN,NaN,0.320,NaN,NaN,NaN,NaN,0.093,NaN,NaN,NaN,0.974,NaN,AFG,1951
AFG_1952,NaN,NaN,NaN,0.324,NaN,NaN,NaN,NaN,0.082,NaN,NaN,NaN,0.974,NaN,AFG,1952
AFG_1953,NaN,NaN,NaN,0.316,NaN,NaN,NaN,NaN,0.081,NaN,NaN,NaN,0.974,NaN,AFG,1953
AFG_1954,NaN,NaN,NaN,0.291,NaN,NaN,NaN,NaN,0.090,NaN,NaN,NaN,0.974,NaN,AFG,1954
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZWE_2016,NaN,37.060849,60.254732,0.392,0.713546,NaN,-0.9568,NaN,0.175,0.295787,NaN,68.004509,1.670,0.61,ZWE,2016
ZWE_2017,NaN,39.095571,59.988293,0.434,0.710000,NaN,-0.9524,31.70614,0.215,NaN,78.38,65.626222,2.274,0.64,ZWE,2017
ZWE_2018,94.53,42.145474,60.749835,0.523,0.730000,6.346832,-0.9291,32.58448,0.195,0.301143,NaN,65.229449,1.849,NaN,ZWE,2018


## Step 4: Link DEC and J2SR Data

In [82]:
dec_j2sr = pd.merge(dec_meta, j2sr_pivot, how='left', left_on = 'country_year', right_index=True)
dec_j2sr.head()

,Abstract,Ancillary_Data,Bibliographic_Type,ContentType,Contract_Grant_Number,Credit,Date_Resource_Created,Description,Descriptors_Topical,Descriptors_Geographic,...,Roadmap Metric: Export Sophistication,"Roadmap Metric: Information & Communication Technology (ICT) Adoption (1-100, higher is better)","Roadmap Metric: Liberal Democracy (0-1, higher is better)","Roadmap Metric: Open Government (0-1, higher is stronger rule of law)",Roadmap Metric: Poverty Rate ($5/Day) (percentage),"Roadmap Metric: Safety & Security (0-100, higher is better)","Roadmap Metric: Social Group Equality (0-4, higher is better)","Roadmap Metric: Tax System Effectiveness (0-1, higher is better)",iso,year_y
Unique_ID,,,,,,,,,,,,,,,,,,,,,
PD-AAB-495-E1,NaN,Evaluation period: 22 Mar 1972-12 Jul 1976,Final Evaluation Report,Documents,NaN,NaN,7/1/1977 0:00,NaN,Management education ~|~_©_~|~ Graduate education ~|~_©_~|~ Limited data,Central America,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
PD-AAP-604,NaN,Evaluation period: 25 Jul 1972-10 Apr 1979,Final Evaluation Report,Documents,NaN,NaN,6/1/1979 0:00,NaN,Low cost housing ~|~_©_~|~ Minimum shelter housing ~|~_©_~|~ Housing ~|~_©_~|~ Earthquakes ~|~_©...,Peru,...,NaN,NaN,0.048,NaN,NaN,NaN,1.304,NaN,PER,1979
PD-AAB-088-A1,NaN,NaN,Special Evaluation,Documents,AID/TA-C-1469,NaN,2/1/1979 0:00,NaN,Road construction ~|~_©_~|~ Roads ~|~_©_~|~ Rural areas,Honduras,...,NaN,NaN,0.061,NaN,NaN,NaN,0.696,NaN,HND,1979
PD-AAA-880-D1,NaN,Evaluation period: 22 Jan 1979-9 Feb 1979,Final Evaluation Report,Documents,AID/LAC-C-1313,NaN,2/1/1979 0:00,NaN,Health surveys ~|~_©_~|~ Demographic research ~|~_©_~|~ MIGRATION ~|~_©_~|~ Samples ~|~_©_~|~ Su...,El Salvador,...,NaN,NaN,0.041,NaN,NaN,NaN,0.631,NaN,SLV,1979
PD-AAT-461,NaN,Evaluation period: 1 Jan1972-31 Dec1976,Special Evaluation,Documents,AID/pha/C-1100,NaN,7/1/1977 0:00,NaN,Condoms ~|~_©_~|~ Sterilization (birth control) ~|~_©_~|~ Oral contraceptives ~|~_©_~|~ Family p...,Thailand,...,NaN,NaN,0.083,NaN,NaN,NaN,0.353,NaN,THA,1977


In [83]:
dec_j2sr.to_csv('dec_j2sr_trial.csv')